#### 3.8.1.4 daum_news_list_final

In [33]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

import re
import string


In [34]:
# db 저장 함수
def db_save(news_list):
    with sqlite3.connect(os.path.join('.','DaumNewsDB')) as con:
        try:
            news_list.to_sql(name='NEWS_LIST',con = con, index = False, if_exists='append')
        except Exception as e:
            print(str(e))
        
        print(len(news_list), '건 저장완료')

In [35]:
# db 확인 함수
def db_select():
    with sqlite3.connect(os.path.join('.','DaumNewsDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            news_list = pd.read_sql(query, con=con)
        except Exception as e:
            print(str(e))
        
        return news_list

In [36]:
# db 삭제 함수
def db_delete():
    with sqlite3.connect(os.path.join('.','DaumNewsDB')) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [37]:
# 상세페이지 내용가져오기
def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

In [58]:
# 20210715 10:48에는 23페이지까지 생성되어있음
# page의 값을 23미만으로 설정
page = 20
max_page = 0
REG_DATE = '20210715'

In [59]:
while(True):
    # url, 제목, 본문내용을 df_list에 저장
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)

    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({'URL' : [url],'TITLE':[a.text],'ARTICLE' : [article]})
        df_list.append(df)   

    # df_list가 있을경우 db에 저장
    if df_list:
        df_10 = pd.concat(df_list)
        db_save(df_10)

    # 페이지 번호 중에 max 페이지 가져오기
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num
        except:
            pass

    # 마지막 페이지 여부 확인
    # 이전 페이지 이동 또는 다음 페이지 이동이 없고 page가 max_page보다 크면 종료
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')
    if (len(span) <= 0) & (page>max_page):
        break
    else:
        page = page + 1

    time.sleep(1)

15 건 저장완료
15 건 저장완료
15 건 저장완료
14 건 저장완료


In [60]:
print(db_select())

                                       URL  \
0   https://v.daum.net/v/20210715075913740   
1   https://v.daum.net/v/20210715075840729   
2   https://v.daum.net/v/20210715075816718   
3   https://v.daum.net/v/20210715075045617   
4   https://v.daum.net/v/20210715074149464   
5   https://v.daum.net/v/20210715073226312   
6   https://v.daum.net/v/20210715072633236   
7   https://v.daum.net/v/20210715072602229   
8   https://v.daum.net/v/20210715072237204   
9   https://v.daum.net/v/20210715072008171   
10  https://v.daum.net/v/20210715071343081   
11  https://v.daum.net/v/20210715071302073   
12  https://v.daum.net/v/20210715070503977   
13  https://v.daum.net/v/20210715070157916   
14  https://v.daum.net/v/20210715070023864   
15  https://v.daum.net/v/20210715070020861   
16  https://v.daum.net/v/20210715064501695   
17  https://v.daum.net/v/20210715064021667   
18  https://v.daum.net/v/20210715064016664   
19  https://v.daum.net/v/20210715064003656   
20  https://v.daum.net/v/202107150